# Création des graphiques

Imports


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects
from scipy import stats

Define global variables

In [ ]:
COULEURS_DPE = {
    'A': '#00A651',
    'B': '#50B847',
    'C': '#C8D220',
    'D': '#FDEE00',
    'E': '#FEB700',
    'F': '#F0832A',
    'G': '#ED1C24'
}

SEUILS_ALTITUDE = [600, 1200, 1800, 2500]

LABELS_ALTITUDE = [
    "0-600m (Vallée)",           # Zones basses : Annecy, Thonon-les-Bains
    "600-1200m (Colline)",       # Moyenne montagne
    "1200-1800m (Montagne)",     # Haute montagne habitée
    "1800-2500m (Haute montagne)",  # Limite des habitations permanentes
    ">2500m (Très haute montagne)"  # Zones peu habitées
]

COULEURS_ALTITUDE = {
    "0-600m (Vallée)": '#27ae60',
    "600-1200m (Colline)": '#f39c12',
    "1200-1800m (Montagne)": '#e74c3c',
    "1800-2500m (Haute montagne)": '#9b59b6',
    ">2500m (Très haute montagne)": '#34495e'
}

TAILLE_ECHANTILLON_SCATTER = 10000

PRIX_ELECTRICITE = 0.20  # €/kWh
SURFACE_REFERENCE = 70  # m²

## Read data

Loads data

In [ ]:
df = pd.read_csv('../web/application/datasets/logements_74.csv')

Select columns

In [ ]:
selected_columns = [
    'numero_dpe',                          # Identifiant unique du DPE
    'code_insee_ban',                      # Code commune
    'nom_commune_ban',                     # Nom de la commune
    'etiquette_dpe',                       # Classe énergétique (A à G)
    'conso_5_usages_par_m2_ep',            # Consommation en kWh/m²/an
    'surface_habitable_logement',          # Surface en m² (pour calculs de coûts)
    'type_batiment',                       # Maison ou appartement
    'periode_construction',                # Époque de construction
    'periode_categorie',                   # Categorie de periode de construction
    'type_energie_principale_chauffage',   # Type de chauffage utilisé
    'altitude_moyenne',                    # Altitude moyenne (déjà dans le DPE)
    'classe_altitude',                     # Classe d'altitude
    'categorie_dpe'                        # Categorie DPE simplifiées
]

In [ ]:
df = df[selected_columns]

## Scatter Plot

In [ ]:
df_valide = df[df['altitude_moyenne'].notna()]
df_plot = df_valide.sample(n=TAILLE_ECHANTILLON_SCATTER, random_state=0)

Linear regression

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(
    df_plot['altitude_moyenne'], 
    df_plot['conso_5_usages_par_m2_ep']
)

r_squared = r_value ** 2

altitude_min = df_plot['altitude_moyenne'].min()
altitude_max = df_plot['altitude_moyenne'].max()
x_regression = np.array([altitude_min, altitude_max])
y_regression = slope * x_regression + intercept

Draw plot

In [ ]:
fig = px.scatter(
    df_plot,
    x='altitude_moyenne',
    y='conso_5_usages_par_m2_ep',
    color='etiquette_dpe',
    color_discrete_map=COULEURS_DPE,
    category_orders={'etiquette_dpe': ['A', 'B', 'C', 'D', 'E', 'F', 'G']},  # Ordre croissant des étiquettes DPE
    opacity=0.6,
    labels={
        'altitude_moyenne': 'Altitude moyenne de la commune (m)',
        'conso_5_usages_par_m2_ep': 'Consommation énergétique (kWh/m²/an)',
        'etiquette_dpe': 'Étiquette DPE'
    },
    hover_data={
        'altitude_moyenne': ':.0f',
        'conso_5_usages_par_m2_ep': ':.1f',
        'nom_commune_ban': True,
        'type_batiment': True,
        'periode_construction': True,
        'etiquette_dpe': False  # Déjà visible via la couleur
    },
    title=f'<b>Impact de l\'altitude sur la consommation énergétique</b><br>'
          f'<sub>Mountain Energy Score - Département Haute-Savoie (74)</sub>'
)

In [ ]:
fig.add_trace(
    graph_objects.Scatter(
        x=x_regression,
        y=y_regression,
        mode='lines',
        name=f'Régression linéaire<br>(+{slope*100:.1f} kWh/m²/an par 100m)',
        line=dict(color='#e74c3c', width=3, dash='dash'),
        hovertemplate='<b>Tendance linéaire</b><br>Altitude: %{x:.0f}m<br>Consommation estimée: %{y:.1f} kWh/m²/an<extra></extra>'
    )
)

Add altitude zones

In [ ]:
zones = [
    (SEUILS_ALTITUDE[0], "lightgreen", "Vallée"),
    (SEUILS_ALTITUDE[1], "indigo", "Colline"),
    (SEUILS_ALTITUDE[2], "lightcoral", "Montagne")
]

altitude_debut = altitude_min
for seuil, couleur, nom in zones:
    if altitude_debut < seuil:
        fig.add_vrect(
            x0=altitude_debut, 
            x1=min(seuil, altitude_max),
            fillcolor=couleur, 
            opacity=0.1, 
            line_width=0,
            annotation_text=nom, 
            annotation_position="top left"
        )
        altitude_debut = seuil

Customize layout

In [ ]:
fig.update_layout(
    # Dimensions pour une bonne lisibilité
    width=1200,
    height=700,
    
    # Style de fond
    plot_bgcolor='white',
    paper_bgcolor='white',
    
    # Configuration du titre
    title={
        'font': {'size': 22, 'family': 'Arial, sans-serif', 'color': '#2c3e50'},
        'x': 0.5,
        'xanchor': 'center'
    },
    
    # Configuration des axes
    xaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='#ecf0f1',
        zeroline=False,
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e')
    ),
    yaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='#ecf0f1',
        zeroline=False,
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e')
    ),
    
    # Configuration de la légende
    legend=dict(
    title=dict(text='<b>Étiquette DPE</b>', font=dict(size=12)),
    orientation='v',
    yanchor='top',
    y=0.98,
    xanchor='right',
    x=0.98,
    bgcolor='rgba(255, 255, 255, 0.9)',
    bordercolor='#bdc3c7',
    borderwidth=1,
    font=dict(size=10),
    ),
    
    # Mode d'interaction au survol
    hovermode='closest',
    hoverlabel=dict(
        bgcolor='white',
        font_size=11,
        font_family='Arial, sans-serif'
    ),
    
    # Marges pour l'annotation en bas
    margin=dict(l=80, r=80, t=100, b=140)
)

In [ ]:
fig.add_annotation(
    text=f"<b>📊 Insight clé :</b> Chaque 100m d'altitude supplémentaire augmente "
         f"la consommation de <b>{slope*100:.1f} kWh/m²/an</b> en moyenne "
         f"<i>(R² = {r_squared:.3f}, p < 0.001)</i>. "
         f"Cette corrélation significative démontre l'impact direct de l'altitude sur les besoins énergétiques.",
    xref="paper", yref="paper",
    x=0.5, y=-0.17,
    xanchor='center', yanchor='top',
    showarrow=False,
    bgcolor='rgba(255, 243, 205, 0.95)',
    bordercolor='#f39c12',
    borderwidth=2,
    borderpad=10,
    font=dict(size=11, family='Arial, sans-serif', color='#34495e')
)

## Boxplot

In [ ]:
df_valide = df[
    (df['altitude_moyenne'].notna()) & 
    (df['classe_altitude'].notna()) &
    (df['conso_5_usages_par_m2_ep'] <= 1000)  # Limite supérieure réaliste
]

Classes

In [ ]:
ordre_tranches = [label for label in LABELS_ALTITUDE if label in df_valide['classe_altitude'].unique()]

In [ ]:
stats_par_tranche = []

for tranche in ordre_tranches:
    donnees_tranche = df_valide[df_valide['classe_altitude'] == tranche]['conso_5_usages_par_m2_ep']
    
    stats = {
        'tranche': tranche,
        'effectif': len(donnees_tranche),
        'mediane': donnees_tranche.median(),
        'moyenne': donnees_tranche.mean(),
        'q1': donnees_tranche.quantile(0.25),
        'q3': donnees_tranche.quantile(0.75),
        'min': donnees_tranche.min(),
        'max': donnees_tranche.max()
    }
    stats_par_tranche.append(stats)

ecart_vallee_montagne = stats_par_tranche[-1]['mediane'] - stats_par_tranche[0]['mediane']
pct_augmentation = (ecart_vallee_montagne / stats_par_tranche[0]['mediane']) * 100

Draw plot

In [ ]:
fig = graph_objects.Figure()

# Ajout d'un boxplot pour chaque tranche d'altitude
for tranche in ordre_tranches:
    donnees_tranche = df_valide[df_valide['classe_altitude'] == tranche]['conso_5_usages_par_m2_ep']
    
    fig.add_trace(graph_objects.Box(
        y=donnees_tranche,
        name=tranche,
        marker_color=COULEURS_ALTITUDE.get(tranche, '#95a5a6'),
        boxmean='sd',  # Affiche aussi la moyenne avec écart-type
        hovertemplate=(
            '<b>%{fullData.name}</b><br>' +
            'Médiane: %{median:.0f} kWh/m²/an<br>' +
            'Q1: %{q1:.0f} kWh/m²/an<br>' +
            'Q3: %{q3:.0f} kWh/m²/an<br>' +
            'Min: %{min:.0f} kWh/m²/an<br>' +
            'Max: %{max:.0f} kWh/m²/an<br>' +
            '<extra></extra>'
        )
    ))

Customize layout

In [ ]:
fig.update_layout(
    # Titre
    title={
        'text': (
            '<b>Distribution de la consommation énergétique par tranche d\'altitude</b><br>'
            '<sub>Mountain Energy Score - Département Haute-Savoie (74)</sub>'
        ),
        'font': {'size': 22, 'family': 'Arial, sans-serif', 'color': '#2c3e50'},
        'x': 0.5,
        'xanchor': 'center'
    },
    
    # Axes
    xaxis=dict(
        title='<b>Tranche d\'altitude</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e'),
        showgrid=False
    ),
    yaxis=dict(
        title='<b>Consommation énergétique (kWh/m²/an)</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#ecf0f1',
        zeroline=False
    ),
    
    # Style
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=False,  # Pas de légende nécessaire (noms sur l'axe X)
    
    # Dimensions
    width=1200,
    height=700,
    
    # Marges pour l'annotation
    margin=dict(l=80, r=80, t=100, b=160)
)

In [ ]:
fig.add_annotation(
    text=(
        f"<b>📊 Insight clé :</b> La consommation médiane augmente de "
        f"<b>{ecart_vallee_montagne:.0f} kWh/m²/an</b> entre la vallée "
        f"({stats_par_tranche[0]['mediane']:.0f} kWh/m²/an) et la haute montagne "
        f"({stats_par_tranche[-1]['mediane']:.0f} kWh/m²/an), soit une hausse de "
        f"<b>{pct_augmentation:.1f}%</b>.<br>"
        f"La dispersion augmente également avec l'altitude, révélant une plus grande hétérogénéité des situations en montagne."
        ),
        xref="paper", yref="paper",
        x=0.5, y=-0.19,
        xanchor='center', yanchor='top',
        showarrow=False,
        bgcolor='rgba(255, 243, 205, 0.95)',
        bordercolor='#f39c12',
        borderwidth=2,
        borderpad=10,
        font=dict(size=11, family='Arial, sans-serif', color='#34495e')
    )

## Barplot

In [ ]:
df_valide = df[
    (df['altitude_moyenne'].notna()) & 
    (df['classe_altitude'].notna()) &
    (df['conso_5_usages_par_m2_ep'] <= 1000)
]

Classes

In [ ]:
ordre_tranches = [label for label in LABELS_ALTITUDE if label in df_valide['classe_altitude'].unique()]

In [ ]:
stats_tranches = []

for tranche in ordre_tranches:
    donnees_tranche = df_valide[df_valide['classe_altitude'] == tranche]['conso_5_usages_par_m2_ep']
    
    conso_moyenne = donnees_tranche.mean()
    effectif = len(donnees_tranche)
    
    stats_tranches.append({
        'tranche': tranche,
        'conso_moyenne': conso_moyenne,
        'effectif': effectif
    })

In [ ]:
conso_reference = stats_tranches[0]['conso_moyenne']

# Calcul du surcoût pour chaque tranche
for stats in stats_tranches:
    # Différence de consommation par rapport à la vallée (kWh/m²/an)
    ecart_conso = stats['conso_moyenne'] - conso_reference
    
    # Surcoût annuel pour un logement de référence (€/an)
    surcout_annuel = ecart_conso * SURFACE_REFERENCE * PRIX_ELECTRICITE
    
    stats['ecart_conso'] = ecart_conso
    stats['surcout_annuel'] = surcout_annuel

In [ ]:
montagne = stats_tranches[-2]
haute_montagne = stats_tranches[-1]

Draw plot

In [ ]:
# Préparation des données pour le graphique
tranches_noms = [s['tranche'] for s in stats_tranches]
surcouts = [s['surcout_annuel'] for s in stats_tranches]
consos = [s['conso_moyenne'] for s in stats_tranches]
effectifs = [s['effectif'] for s in stats_tranches]

couleurs_barres = [COULEURS_ALTITUDE.get(t, '#95a5a6') for t in tranches_noms]

In [ ]:
fig = graph_objects.Figure()

fig.add_trace(graph_objects.Bar(
    x=tranches_noms,
    y=surcouts,
    marker_color=couleurs_barres,
    text=[f"<b>{s:.0f} €/an</b><br>({c:.0f} kWh/m²/an)<br>n = {e:,}" 
          for s, c, e in zip(surcouts, consos, effectifs)],
    textposition='inside',  
    textfont=dict(size=12, color='white', family='Arial, sans-serif'),  
    hovertemplate=(
        '<b>%{x}</b><br>' +
        'Surcoût annuel: %{y:.0f} €/an<br>' +
        'Consommation: %{customdata[0]:.0f} kWh/m²/an<br>' +
        'Effectif: %{customdata[1]:,} logements<br>' +
        '<extra></extra>'
    ),
    customdata=[[c, e] for c, e in zip(consos, effectifs)]
))

Customize layout

In [ ]:
surcout_max = max(surcouts)
tranche_max = tranches_noms[surcouts.index(surcout_max)]
conso_max = consos[surcouts.index(surcout_max)]

In [ ]:
fig.update_layout(
    # Titre
    title={
        'text': (
            '<b>Surcoût énergétique annuel par tranche d\'altitude</b><br>'
            f'<sub>Mountain Energy Score - Département Haute-Savoie (74) - '
            f'Logement de référence : {SURFACE_REFERENCE}m², prix électricité {PRIX_ELECTRICITE}€/kWh</sub>'
        ),
        'font': {'size': 22, 'family': 'Arial, sans-serif', 'color': '#2c3e50'},
        'x': 0.5,
        'xanchor': 'center'
    },
    
    # Axes
    xaxis=dict(
        title='<b>Tranche d\'altitude</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e'),
        showgrid=False
    ),
    yaxis=dict(
        title='<b>Surcoût énergétique annuel (€/an)</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#ecf0f1',
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='#34495e'
    ),
    
    # Style
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=False,
    
    # Dimensions
    width=1200,
    height=700,
    
    # Marges
    margin=dict(l=80, r=80, t=120, b=160)
)

In [ ]:
fig.add_annotation(
    text=(
        f"<b>💰 Insight clé :</b> Vivre en {tranche_max.lower()} coûte <b>+{surcout_max:.0f}€/an</b> "
        f"de plus en énergie par rapport à la vallée pour un logement de {SURFACE_REFERENCE}m².<br>"
        f"Cela représente l'équivalent de <b>{surcout_max/12:.0f}€ par mois</b> de facture énergétique supplémentaire, "
        f"soit un budget non négligeable pour les ménages vivant en altitude."
    ),
    xref="paper", yref="paper",
    x=0.5, y=-0.19,
    xanchor='center', yanchor='top',
    showarrow=False,
    bgcolor='rgba(255, 243, 205, 0.95)',
    bordercolor='#f39c12',
    borderwidth=2,
    borderpad=10,
    font=dict(size=11, family='Arial, sans-serif', color='#34495e')
)

## Barre DPE

In [ ]:
df_valide = df[
    (df['altitude_moyenne'].notna()) & 
    (df['classe_altitude'].notna()) &
    (df['etiquette_dpe'].isin(['A', 'B', 'C', 'D', 'E', 'F', 'G']))
]

Classes

In [ ]:
ordre_tranches = [label for label in LABELS_ALTITUDE if label in df_valide['classe_altitude'].unique()]

In [ ]:
stats_par_tranche = []

for tranche in ordre_tranches:
    donnees_tranche = df_valide[df_valide['classe_altitude'] == tranche]
    total = len(donnees_tranche)
    
    # Comptage par catégorie
    categories = donnees_tranche['categorie_dpe'].value_counts()
    
    stats = {
        'tranche': tranche,
        'total': total,
        'pct_bons': (categories.get('Bons (A-B)', 0) / total * 100),
        'pct_moyens': (categories.get('Moyens (C-D)', 0) / total * 100),
        'pct_mediocres': (categories.get('Médiocres (E)', 0) / total * 100),
        'pct_passoires': (categories.get('Passoires (F-G)', 0) / total * 100),
        'nb_passoires': categories.get('Passoires (F-G)', 0)
    }
    
    stats_par_tranche.append(stats)

In [ ]:
pct_vallee = stats_par_tranche[0]['pct_passoires']
pct_montagne_max = max([s['pct_passoires'] for s in stats_par_tranche])
ratio = pct_montagne_max / pct_vallee if pct_vallee > 0 else 0

Draw plot

In [ ]:
tranches_noms = [s['tranche'] for s in stats_par_tranche]

In [ ]:
fig = graph_objects.Figure()

# Ordre d'empilement : du meilleur au pire (A-B en bas, F-G en haut)
fig.add_trace(graph_objects.Bar(
    name='Bons DPE (A-B)',
    x=tranches_noms,
    y=[s['pct_bons'] for s in stats_par_tranche],
    marker_color='#27ae60',
    text=[f"{s['pct_bons']:.1f}%" for s in stats_par_tranche],
    textposition='inside',
    textfont=dict(color='white', size=11),
    hovertemplate='<b>%{x}</b><br>Bons DPE (A-B): %{y:.1f}%<extra></extra>'
))

fig.add_trace(graph_objects.Bar(
    name='DPE moyens (C-D)',
    x=tranches_noms,
    y=[s['pct_moyens'] for s in stats_par_tranche],
    marker_color='#f39c12',
    text=[f"{s['pct_moyens']:.1f}%" for s in stats_par_tranche],
    textposition='inside',
    textfont=dict(color='white', size=11),
    hovertemplate='<b>%{x}</b><br>DPE moyens (C-D): %{y:.1f}%<extra></extra>'
))

fig.add_trace(graph_objects.Bar(
    name='DPE médiocres (E)',
    x=tranches_noms,
    y=[s['pct_mediocres'] for s in stats_par_tranche],
    marker_color='#e67e22',
    text=[f"{s['pct_mediocres']:.1f}%" for s in stats_par_tranche],
    textposition='inside',
    textfont=dict(color='white', size=11),
    hovertemplate='<b>%{x}</b><br>DPE médiocres (E): %{y:.1f}%<extra></extra>'
))

fig.add_trace(graph_objects.Bar(
    name='PASSOIRES THERMIQUES (F-G)',
    x=tranches_noms,
    y=[s['pct_passoires'] for s in stats_par_tranche],
    marker_color='#e74c3c',
    text=[f"<b>{s['pct_passoires']:.1f}%</b><br>({s['nb_passoires']:,})" for s in stats_par_tranche],
    textposition='inside',
    textfont=dict(color='white', size=12, family='Arial, sans-serif'),
    hovertemplate='<b>%{x}</b><br>Passoires (F-G): %{y:.1f}%<extra></extra>'
))

Customize layout

In [ ]:
fig.update_layout(
    # Empilement à 100%
    barmode='stack',
    
    # Titre
    title={
        'text': (
            '<b>Concentration des passoires thermiques selon l\'altitude</b><br>'
            f'<sub>Mountain Energy Score - Département Haute-Savoie (74)</sub>'
        ),
        'font': {'size': 22, 'family': 'Arial, sans-serif', 'color': '#2c3e50'},
        'x': 0.5,
        'xanchor': 'center'
    },
    
    # Axes
    xaxis=dict(
        title='<b>Tranche d\'altitude</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e')
    ),
    yaxis=dict(
        title='<b>Répartition des DPE (%)</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e'),
        range=[0, 100],
        ticksuffix='%'
    ),
    
    # Style
    plot_bgcolor='white',
    paper_bgcolor='white',
    
    # Légende
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5,
        font=dict(size=11)
    ),
    
    # Dimensions
    width=1200,
    height=700,
    
    # Marges
    margin=dict(l=80, r=80, t=140, b=160)
)

In [ ]:
fig.add_annotation(
    text=(
        f"<b>🔥 Insight clé :</b> Les passoires thermiques (F/G) représentent "
        f"<b>{pct_vallee:.1f}%</b> des logements en vallée contre <b>{pct_montagne_max:.1f}%</b> en montagne,<br>"
        f"soit <b>×{ratio:.1f}</b> plus de risque d'avoir un logement énergivore en altitude. "
        f"Cette concentration révèle un enjeu majeur de rénovation énergétique en zone de montagne."
    ),
    xref="paper", yref="paper",
    x=0.5, y=-0.19,
    xanchor='center', yanchor='top',
    showarrow=False,
    bgcolor='rgba(255, 243, 205, 0.95)',
    bordercolor='#f39c12',
    borderwidth=2,
    borderpad=10,
    font=dict(size=11, family='Arial, sans-serif', color='#34495e')
)

## Periode de construction

In [ ]:
df_valide = df[
    (df['altitude_moyenne'].notna()) & 
    (df['classe_altitude'].notna()) &
    (df['conso_5_usages_par_m2_ep'] <= 1000) &
    (df['periode_categorie'].notna())
]

Compute heatmap

In [ ]:
ordre_tranches = [label for label in LABELS_ALTITUDE if label in df_valide['classe_altitude'].unique()]

# Ordre chronologique des périodes
ordre_periodes = ['Avant 1975', '1975-2000', '2001-2012', 'Après 2012']
ordre_periodes = [p for p in ordre_periodes if p in df_valide['periode_categorie'].unique()]

In [ ]:
tableau_conso = pd.DataFrame(index=ordre_periodes, columns=ordre_tranches)
tableau_effectifs = pd.DataFrame(index=ordre_periodes, columns=ordre_tranches)

for periode in ordre_periodes:
    for tranche in ordre_tranches:
        mask = (df_valide['periode_categorie'] == periode) & (df_valide['classe_altitude'] == tranche)
        logements = df_valide[mask]
        
        if len(logements) >= 10:  # Seuil minimum
            conso_moyenne = logements['conso_5_usages_par_m2_ep'].mean()
            effectif = len(logements)
            
            tableau_conso.loc[periode, tranche] = conso_moyenne
            tableau_effectifs.loc[periode, tranche] = effectif

Cross analysis

In [ ]:
tableau_conso_numeric = tableau_conso.apply(pd.to_numeric, errors='coerce')
max_conso = tableau_conso_numeric.max().max()
max_position = tableau_conso_numeric.stack().idxmax()

pire_periode = max_position[0]
pire_tranche = max_position[1]

# Comparaison avec le meilleur
min_conso = tableau_conso_numeric.min().min()
min_position = tableau_conso_numeric.stack().idxmin()

meilleur_periode = min_position[0]
meilleur_tranche = min_position[1]

ecart = max_conso - min_conso
pct_ecart = (ecart / max_conso) * 100

Draw plot

In [ ]:
z_data = tableau_conso_numeric.values

text_data = []
for i, periode in enumerate(tableau_conso.index):
    row_text = []
    for j, tranche in enumerate(tableau_conso.columns):
        conso = tableau_conso.iloc[i, j]
        effectif = tableau_effectifs.iloc[i, j]
        if pd.notna(conso):
            # Choix de la couleur de texte selon le fond de la cellule pour garantir la lisibilité
            # Seuil 220 kWh/m²/an : correspond approximativement à la limite entre fonds clairs et foncés
            couleur = "black" if conso > 220 else "white"
            row_text.append(
                f"<span style='color:{couleur}'><b>{conso:.0f}</b> kWh/m²/an<br>({int(effectif):,})</span>"
            )
        else:
            row_text.append("")
    text_data.append(row_text)

In [ ]:
fig = graph_objects.Figure(data=graph_objects.Heatmap(
    z=z_data,
    x=ordre_tranches,
    y=ordre_periodes,
    text=text_data,
    texttemplate='%{text}',
    textfont={"size": 10, "family": "Arial, sans-serif"},
    colorscale='RdYlGn_r',  # Rouge (mauvais) → Jaune → Vert (bon), inversé
    colorbar=dict(
        title=dict(
            text="Consommation<br>(kWh/m²/an)",
            side="right"
        ),
        thickness=15,
        len=0.7
    ),
    hovertemplate=(
        '<b>%{y}</b><br>' +
        '%{x}<br>' +
        'Consommation: <b>%{z:.0f} kWh/m²/an</b><br>' +
        '<extra></extra>'
    )
))

Customize layout

In [ ]:
fig.update_layout(
    # Titre
    title={
        'text': (
            '<b>Double peine : ancienneté du bâti × altitude</b><br>'
            '<sub>Mountain Energy Score - Département Haute-Savoie (74)</sub>'
        ),
        'font': {'size': 22, 'family': 'Arial, sans-serif', 'color': '#2c3e50'},
        'x': 0.5,
        'xanchor': 'center'
    },
    
    # Axes
    xaxis=dict(
        title='<b>Tranche d\'altitude</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=10, color='#34495e'),
        side='bottom',
        tickangle=0
    ),
    yaxis=dict(
        title='<b>Période de construction</b>',
        title_font=dict(size=13, color='#34495e'),
        tickfont=dict(size=11, color='#34495e')
    ),
    
    # Style
    plot_bgcolor='white',
    paper_bgcolor='white',
    
    # Dimensions
    width=1200,
    height=700,
    
    # Marges
    margin=dict(l=120, r=150, t=120, b=160)
)

In [ ]:
fig.add_annotation(
    text=(
        f"<b>🔥 Insight clé :</b> Les logements <b>{pire_periode}</b> en <b>{pire_tranche.lower()}</b> "
        f"consomment jusqu'à <b>{max_conso:.0f} kWh/m²/an</b>, soit <b>{pct_ecart:.0f}% de plus</b> "
        f"que les logements récents en vallée ({min_conso:.0f} kWh/m²/an).<br>"
        f"Cette double peine (ancien bâti + altitude) révèle une priorité absolue pour les politiques "
        f"de rénovation énergétique : <b>{int(tableau_effectifs.loc[pire_periode, pire_tranche]):,} logements</b> "
        f"sont concernés dans cette seule catégorie."
    ),
    xref="paper", yref="paper",
    x=0.5, y=-0.19,
    xanchor='center', yanchor='top',
    showarrow=False,
    bgcolor='rgba(255, 243, 205, 0.95)',
    bordercolor='#f39c12',
    borderwidth=2,
    borderpad=10,
    font=dict(size=11, family='Arial, sans-serif', color='#34495e')
)